In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
class DopplerVisualizer:
    def __init__(self):
       
        self.observer_pos = 0
        self.source_pos = 5.0
        self.wave_speed = 8.0  # Speed of wave propagation
        self.compression_threshold = 25  # Distance where wave compression becomes noticeable
        
        # Create figure
        self.fig = go.Figure()
        
        # Setup layout
        self.fig.update_layout(
            title='Doppler Effect Visualization',
            xaxis=dict(
                range=[-50, 50],
                title='Position (m)',
                zeroline=True,
                gridwidth=1,
            ),
            yaxis=dict(
                range=[-50, 50],
                title='Position (m)',
                zeroline=True,
                gridwidth=1,
                scaleanchor='x',
                scaleratio=1
            ),
            showlegend=True,
            sliders=[{
                'currentvalue': {'prefix': 'Source Position: '},
                'pad': {'t': 50},
                'steps': [{'label': str(pos),
                          'method': 'animate',
                          'args': [[f'frame_{pos}'], {
                              'frame': {'duration': 0, 'redraw': True},
                              'mode': 'immediate',
                          }]}
                         for pos in np.arange(-50, 50.1, 0.5)],
            }],
            updatemenus=[{
                'type': 'buttons',
                'showactive': False,
                'buttons': [{
                    'label': 'Play',
                    'method': 'animate',
                    'args': [None, {
                        'frame': {'duration': 50, 'redraw': True},
                        'fromcurrent': True,
                        'mode': 'immediate',
                    }],
                }],
            }],
        )

    # Calculate wave front coordinates based on wave source position
    def calculate_wavefront(self, source_x, emission_time, current_time):
 
        theta = np.linspace(0, 2*np.pi, 200)
        
        # Time since emission
        dt = current_time - emission_time
        
        # Source velocity (derived from position change)
        source_velocity = (source_x - self.observer_pos) / 20.0  # Scaled for visualization
        
        # Calculate wave distortion
        r = self.wave_speed * dt
        
        # Calculate doppler shift factor (beta = v/c)
        beta = source_velocity / self.wave_speed * 0.8
        
        # Calculate distorted radius for each angle
        # Using relativistic wave front equation (approximated for visualization)
        r_distorted = r * (1 - beta * np.cos(theta)) / (1 - beta**2)
        
        # Calculate wave front coordinates
        x = source_x - source_velocity * dt + r_distorted * np.cos(theta)
        y = r_distorted * np.sin(theta)
        
        return x, y


    # Subsequent wave generator
    def generate_wave_circles(self, source_x, frame_num):

        circles_data = []
        current_time = frame_num
        
        # Calculate relative velocity
        relative_velocity = (source_x - self.observer_pos)
        approaching = relative_velocity < 0
        
        # Set number of waves as 12

        for i in range(12):
            
            emission_time = current_time - i * 1
                        
            # Generate distorted wave front
            x, y = self.calculate_wavefront(source_x, emission_time, current_time)
            
            # Calculate opacity
            opacity = max(0.1, 0.6 - (i * 0.05))
            
            # Determine color based on distance
            color = 'blue' if -25 <= source_x <= 25 else 'red'
            
            circles_data.append(
                go.Scatter(
                    x=x, y=y,
                    mode='lines',
                    line=dict(
                        color=color,
                        width=1.5,
                    ),
                    opacity=opacity,
                    showlegend=False,
                    hoverinfo='skip'
                )
            )
        
        return circles_data

    # Animate the visualizer
    def create_frames(self):

        frames = []
        source_positions = np.arange(-50, 50.1, 0.5)
        
        for idx, source_x in enumerate(source_positions):
            frame_data = []
            
            # Add observer (static)
            frame_data.append(
                go.Scatter(
                    x=[self.observer_pos],
                    y=[0],
                    mode='markers',
                    marker=dict(
                        size=15,
                        color='green',
                        symbol='circle'
                    ),
                    name='Observer',
                    showlegend=idx == 0
                )
            )
            
            # Add source
            color = 'blue' if -25 <= source_x <= 25 else 'red'
            frame_data.append(
                go.Scatter(
                    x=[source_x],
                    y=[0],
                    mode='markers',
                    marker=dict(
                        size=15,
                        color=color,
                        symbol='circle'
                    ),
                    name='Source',
                    showlegend=idx == 0
                )
            )
            
            # Add distorted wave circles
            frame_data.extend(self.generate_wave_circles(source_x, idx))
            
            # Add effect description
            text = ('Blueshift: Compressed waves,<br>Higher frequency' 
                   if source_x < self.observer_pos 
                   else 'Redshift: Expanded waves,<br>Lower frequency')
            
            frame_data.append(
                go.Scatter(
                    x=[source_x],
                    y=[28],
                    mode='text',
                    text=[text],
                    textfont=dict(
                        color=color,
                        size=14
                    ),
                    showlegend=False,
                    hoverinfo='skip'
                )
            )
            
            frames.append(go.Frame(data=frame_data, name=f'frame_{source_x}'))
        
        return frames

    # Create and display the visualizer
    def run(self):
        # Create initial frame
        initial_data = [
            go.Scatter(
                x=[self.observer_pos],
                y=[0],
                mode='markers',
                marker=dict(size=15, color='green'),
                name='Observer'
            ),
            go.Scatter(
                x=[self.source_pos],
                y=[0],
                mode='markers',
                marker=dict(size=15, color='red'),
                name='Source'
            )
        ]
        
        # Add initial waves
        initial_data.extend(self.generate_wave_circles(self.source_pos, 0))
        
        # Set initial data
        self.fig.add_traces(initial_data)
        
        # Add frames
        self.fig.frames = self.create_frames()
        
        # Show figure
        self.fig.show()

if __name__ == "__main__":
    visualizer = DopplerVisualizer()
    visualizer.run()